In [1]:
# Python libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve, train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score
from sklearn.metrics import classification_report, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve
import warnings
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore') #ignore warning messages 

In [2]:
# data import

train = pd.read_csv('train dataset.csv')
test = pd.read_csv("test dataset.csv")

#data cleaning

train =  train.loc[train['Gender'] != '5']

#separating dependent and independent variables
x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]


x_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

#Encoding Categorical variables
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
x_train = x_train.drop('Gender_Male',axis = 1) #to avoid dummy trap
x_test = x_test.drop('Gender_Male', axis = 1)

from sklearn.preprocessing import LabelEncoder
labelencoder_Y = LabelEncoder()
y_train= labelencoder_Y.fit_transform(y_train)
y_test = labelencoder_Y.transform(y_test)

In [3]:
from catboost import Pool, CatBoostClassifier

cat_features = [0]
train_dataset = Pool(data=x_train,
                     label=y_train,
                     cat_features=cat_features)

eval_dataset = Pool(data=x_test,
                    label=y_test,
                    cat_features=cat_features)

# Initialize CatBoostClassifier
cbc = CatBoostClassifier(iterations=10,
                           learning_rate=0.03,
                           depth=4,
                           l2_leaf_reg = 3,
                           border_count = 3,
                           loss_function='MultiClass')
# Fit model
cbc.fit(train_dataset)

# Get predicted classes
preds_class = cbc.predict(eval_dataset)

cr1 = classification_report(y_test, preds_class, target_names = ['Dependable', 'Extraverted', 'Lively', 'Responsible', 'Serious'])    # Classification Report
print(cr1)

0:	learn: 1.6075005	total: 61.2ms	remaining: 551ms
1:	learn: 1.6062759	total: 75.1ms	remaining: 301ms
2:	learn: 1.6044881	total: 83.3ms	remaining: 194ms
3:	learn: 1.6030802	total: 91.4ms	remaining: 137ms
4:	learn: 1.6020590	total: 95.6ms	remaining: 95.6ms
5:	learn: 1.6015990	total: 100ms	remaining: 66.7ms
6:	learn: 1.6010232	total: 104ms	remaining: 44.8ms
7:	learn: 1.5997619	total: 109ms	remaining: 27.2ms
8:	learn: 1.5982411	total: 113ms	remaining: 12.6ms
9:	learn: 1.5973931	total: 118ms	remaining: 0us
              precision    recall  f1-score   support

  Dependable       0.25      0.05      0.08        21
 Extraverted       0.48      0.77      0.59        77
      Lively       0.33      0.04      0.07        24
 Responsible       1.00      0.03      0.05        40
     Serious       0.62      0.74      0.67       153

    accuracy                           0.56       315
   macro avg       0.54      0.32      0.29       315
weighted avg       0.59      0.56      0.49       315



In [4]:
model = cbc

In [5]:
import joblib #for gui
import pickle #for gui
pickle.dump(model, open("personality.pickle", "wb"))

In [21]:
from tkinter import*
from PIL import ImageTk, Image
import pickle
root=Tk() #create window
root.geometry("700x600") #size of window
root.configure(background="lightblue") #color of window

def model(Age, openness, neuroticism, conscientiousness, agreeableness, extraversion, Gender_Female):
    loaded_model1 = pickle.load(open("personality.pickle", "rb"))
    prediction = loaded_model1.predict([[Age, openness, neuroticism, conscientiousness, agreeableness, extraversion, Gender_Female]])  #order important
    if prediction == [0]:
        notif1.configure(text="You are Dependable")
    elif prediction == [1]:
        notif1.configure(text="You are an Extrovert")
    elif prediction == [2]:
        notif1.configure(text="You are Lively") 
    elif prediction == [3]:
        notif1.configure(text="You are Responsible")
    else:
        notif1.configure(text="You are Serious")
        
Head1 = Label(root, text="AIS Solutions Pvt Ltd", font=('Times New Roman',22,"bold"),bg="orange",fg = 'Black').pack() #square box
Head2 = Label(root,text="Personality Prediction",font=('Cambria',14,'bold'),bg="red",fg = 'white',relief="solid").pack(pady=(10,0)) #spacing
Head3= Label(root,text="Enter values of the following variables to predict your personality",font=('Times New Roman',12,"bold"),bg="light grey").pack(pady=(10,0))
Head4 = Label(root,text=" ",font=('Times New Roman',12,"bold"),bg="turquoise").pack(pady=(10,0))
Head5 = Label(root,text=" ",font=('Times New Roman',12,"bold"),bg="turquoise").pack(pady=(10,0))
Head6= Label(root,text="Rate yourself between 1-8",font=('Times New Roman',12,"bold"),bg="light grey").pack(pady=(10,0))

Age=Label(root,text="Age (in years)",font=('Helvetica',10,'bold'),bg="white",relief="solid",width=18).place(x=30,y=110) #to shift box
Age_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
Age_input.place(x=170,y=110)



openness=Label(root,text="Openness",font=('Helvetica',10,'bold'),bg="white",relief="solid",width=18).place(x=30,y=460)
openness_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
openness_input.place(x=170,y=460)

neuroticism=Label(root,text="Neuroticism (Anxious)",font=('Helvetica',9,'bold'),bg="white",relief="solid",width=18).place(x=30,y=230)
neuroticism_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
neuroticism_input.place(x=170,y=230)

conscientiousness=Label(root,text="Conscientious",font=('Helvetica',9,'bold'),bg="white",relief="solid",width=18).place(x=30,y=290)
conscientiousness_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
conscientiousness_input.place(x=170,y=290)

agreeableness=Label(root,text="Agreeableness",font=('Helvetica',8,'bold'),bg="white",relief="solid",width=18).place(x=30,y=350)
agreeableness_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
agreeableness_input.place(x=170,y=350)

extraversion=Label(root,text="Extraversion (Social)",font=('Helvetica',10,'bold'),bg="white",relief="solid",width=18).place(x=30,y=410)
extraversion_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
extraversion_input.place(x=170,y=410)

Gender_Female=Label(root,text="Gender (F:1, M:0)",font=('Helvetica',10,'bold'),bg="white",relief="solid",width=18).place(x=30,y=150)
Gender_Female_input=Entry(root,bd=2,width="15",font=("Arial sans MS",10))
Gender_Female_input.place(x=170,y=150)


but1 = Button(root,text="Click to know your Personality",width="30", height="2",command=lambda: model(Age_input.get(),openness_input.get(),neuroticism_input.get(),conscientiousness_input.get(),agreeableness_input.get(),extraversion_input.get(),Gender_Female_input.get()))
notif1 = Label(root, text="",font=("Arial sans MS",14,"bold"))

but1.place(x=300,y=550)
notif1.place(x=10,y=500)
root.resizable(0,0)
root.mainloop()